In [ ]:
! pip install openai==0.28

In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 如何加载使用tool
- 加载预制tool的方法
- 几种tool的使用方式
<hr>

langchain预制了大量的tools，基本这些工具能满足大部分需求，https://github.com/langchain-ai/langchain/tree/v0.0.352/docs/docs/integrations/tools

In [2]:
#添加预制工具的方法很简单
from langchain.agents import load_tools
tool_names = [...]
tools = load_tools(tool_names) #使用load方法
#有些tool需要单独设置llm
from langchain.agents import load_tools
tool_names = [...]
llm = ...
tools = load_tools(tool_names, llm=llm) #在load的时候指定llm

ValueError: Got unknown tool Ellipsis

## SerpAPI
最常见的聚合搜索引擎 https://serper.dev/dashboard

In [3]:
from langchain.utilities import SerpAPIWrapper

In [4]:
#serpapi的api key
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"

In [5]:
search = SerpAPIWrapper()

In [6]:
search.run("Obama's first name?")

ValueError: Got error from SerpAPI: Your account has run out of searches.

支持自定义参数，比如将引擎切换到bing，设置搜索语言等

In [ ]:
params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
search = SerpAPIWrapper(params=params)

In [ ]:
search.run("Obama's first name?")

# 使用Dall-E
Dall-E是openai出品的文到图AI大模型

In [ ]:
! pip install opencv-python scikit-image

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
)

In [ ]:
from langchain.agents import initialize_agent, load_tools

tools = load_tools(["dalle-image-generator"])
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description",
    verbose=True
)
output = agent.run("Create an image of a halloween night at a haunted museum")

# Eleven Labs Text2Speech
ElevenLabs 是非常优秀的TTS合成API

In [ ]:
! pip install elevenlabs

In [ ]:
! pip install --upgrade pydantic

插入APIKEY

In [ ]:
import os

os.environ["ELEVEN_API_KEY"] = "23261e4a3b79697822252a505a169863"

工具使用

In [ ]:
from langchain.tools import ElevenLabsText2SpeechTool

text_to_speak = "Hello! 你好! Hola! नमस्ते! Bonjour! こんにちは! مرحبا! 안녕하세요! Ciao! Cześć! Привіт! வணக்கம்!"

tts = ElevenLabsText2SpeechTool(
    voice="Bella",
    text_to_speak=text_to_speak,
    verbose=True
)
tts.name

In [ ]:
speech_file = tts.run(text_to_speak)

In [ ]:
#tts.play(speech_file)
tts.stream_speech(text_to_speak)

# GraphQL
一种api查询语言，类似sql，我们用它来查询奈飞的数据库，查找一下和星球大战相关的电影，API地址https://swapi-graphql.netlify.app/.netlify/functions/index

In [ ]:
! pip install httpx gql > /dev/null

In [ ]:
! pip install gql

In [ ]:
! pip install requests_toolbelt

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.utilities import GraphQLAPIWrapper

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4",
    )

tools = load_tools(
    ["graphql"],
    graphql_endpoint="https://swapi-graphql.netlify.app/.netlify/functions/index",
)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
graphql_fields = """allFilms {
    films {
      title
      director
      releaseDate
      speciesConnection {
        species {
          name
          classification
          homeworld {
            name
          }
        }
      }
    }
  }

"""

suffix = "Search for the titles of all the stawars films stored in the graphql database that has this schema,and answer in chinese:"


agent.run(suffix + graphql_fields)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
To answer this question, I need to query the GraphQL database for all the Star Wars films and their titles. I will use the provided schema to construct the query. After getting the titles, I will translate them into Chinese.

Action: query_graphql
Action Input: query { allFilms { films { title } } }
Observation: "{\n  \"allFilms\": {\n    \"films\": [\n      {\n        \"title\": \"A New Hope\"\n      },\n      {\n        \"title\": \"The Empire Strikes Back\"\n      },\n      {\n        \"title\": \"Return of the Jedi\"\n      },\n      {\n        \"title\": \"The Phantom Menace\"\n      },\n      {\n        \"title\": \"Attack of the Clones\"\n      },\n      {\n        \"title\": \"Revenge of the Sith\"\n      }\n    ]\n  }\n}"
Thought:I have retrieved the titles of all the Star Wars films from the GraphQL database. Now I need to translate these titles into Chinese.

Action: translate
Action Input: ["A New Hope", "The Empire Strikes Back", "Re

'The titles of the Star Wars films are "A New Hope", "The Empire Strikes Back", "Return of the Jedi", "The Phantom Menace", "Attack of the Clones", and "Revenge of the Sith".'